In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
def init_base(shape):
    init_base_value = tf.constant(0.1, shape=shape)
    return tf.Variable(init_base_value)

In [6]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [7]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
def convolution_layer(input_x, shape):
    W = init_weights(shape)
    b = init_base([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [9]:
def full_network_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_base([size])
    return tf.matmul(input_layer,W)+b

In [10]:
x = tf.placeholder(tf.float32, shape = [None, 784])

In [11]:
y_true =tf.placeholder(tf.float32, shape = [None, 10])

In [12]:
x_image = tf.reshape(x, [-1,28,28,1])

In [13]:
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [14]:
y_true

<tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>

In [15]:
conv_1 = convolution_layer(x_image, [6,6,1,32])
conv_1_pooling = max_pool_2by2(conv_1)

In [16]:
conv_2 = convolution_layer(conv_1_pooling, [6,6,32,64])
conv_2_pooling = max_pool_2by2(conv_2)

In [17]:
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(full_network_layer(conv_2_flat, 1024))

In [18]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [19]:
y_pred = full_network_layer(full_one_dropout,10)

In [20]:
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [22]:
init = tf.global_variables_initializer()

In [26]:
saver = tf.train.Saver()

In [27]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')
    saver.save(sess, 'models/my_first_model.ckpt')

Currently on step 0
Accuracy is:
0.1039


Currently on step 100
Accuracy is:
0.79


Currently on step 200
Accuracy is:
0.8866


Currently on step 300
Accuracy is:
0.9137


Currently on step 400
Accuracy is:
0.9261


Currently on step 500
Accuracy is:
0.9388


Currently on step 600
Accuracy is:
0.9465


Currently on step 700
Accuracy is:
0.9515


Currently on step 800
Accuracy is:
0.9557


Currently on step 900
Accuracy is:
0.9579


Currently on step 1000
Accuracy is:
0.9613


Currently on step 1100
Accuracy is:
0.9611


Currently on step 1200
Accuracy is:
0.9647


Currently on step 1300
Accuracy is:
0.9662


Currently on step 1400
Accuracy is:
0.9671


Currently on step 1500
Accuracy is:
0.9679


Currently on step 1600
Accuracy is:
0.9696


Currently on step 1700
Accuracy is:
0.9709


Currently on step 1800
Accuracy is:
0.9728


Currently on step 1900
Accuracy is:
0.9731


Currently on step 2000
Accuracy is:
0.9738


Currently on step 2100
Accuracy is:
0.9746


Currently on step 2200
A

ValueError: Parent directory of models/my_first_model.ckpt doesn't exist, can't save.